<a href="https://colab.research.google.com/github/Proka01/Smith-Waterman-python-CUDA/blob/main/Smith%E2%80%93Waterman_CUDA_projekat3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 KB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 11.0 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp38-cp38-linux_x86_64.whl size=646530 sha256=ba4a3d9e5e4edff73cb06f97f5bb9bc302af854ed8d987bb5480b427e95942f9
  Stored in directory: /root/.cache/pip/wheels/7b/41/0d/7cecb04af969d283ebe4a69579a8b2baec0d010a1ac4159f7e
  Created wheel for pytools: filename=pytools-2022.1.14-py2.py3-none-any.whl size=69870 sha256=5a6e3f67ee62a050287e797ed7edaacdc74d05251f084fb0072f575f9daa9e52
  Stored in directory: /root/.cache/pip/wheels/cb/fc/

In [ ]:
!nvidia-smi

Thu Jan 12 08:09:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np

def cuda_alloc(*args):
  return [cuda.mem_alloc(arg.nbytes) for arg in args]


def to_cuda(*args):
  cuda_ptrs = cuda_alloc(*args)
  for dst, src in zip(cuda_ptrs, args):
     cuda.memcpy_htod(dst, src)
  return cuda_ptrs

g1 = "AGCGA"
g2 = "ACGAA"
gene1 = g1.encode()
gene2 = g2.encode()

gridSize = len(gene1) + 1
grid = np.zeros(shape=(gridSize,gridSize)).astype(dtype=np.int32)
gap = 2
miss = 1
mtch = 1

In [ ]:
#Zadatak 1 - paralelno izvrsavanje

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

mod = SourceModule("""
     __global__ void SmithWaterman(int *grid, char *geneRow, char *geneCol, int match, int miss, int gap, int n)
     {
       int tid = threadIdx.x + 1;   //thread id 1-indexed
       int cnt = 1;             //counts num of cells that thread has processed (thread should procces n cells)

       int start = 1;
       int end = 2*(n-1);

      for(int did = start; did < end; did++) // did => diagonal id
      {
        if(did >= tid && cnt < n)
        {
          char Ai = geneRow[cnt-1];
          char Bi = geneCol[tid-1];

          int left = grid[tid*n + cnt-1];       // grid[tid][did-1];
          int up = grid[(tid-1)*n + cnt];       //grid[tid-1][did];
          int top = grid[(tid-1)*n + cnt-1];    // grid[tid-1][cnt-1];

          int score = (Ai == Bi) ? match : -1*miss;

          int ans = 0;                          //answer for grid[tid][did]
          if(top + score > ans) ans = top + score;
          if(up - gap > ans) ans = up - gap;
          if(left - gap > ans) ans = left - gap;

          grid[tid*n + cnt] = ans;

          cnt++;
        }

        __syncthreads();
      }


     }
   """)

In [ ]:
# kopiranje Host TO Device
grid_cuda = to_cuda(grid)[0]
gene1_cuda = cuda.mem_alloc(len(gene1))
cuda.memcpy_htod(gene1_cuda, gene1)
gene2_cuda = cuda.mem_alloc(len(gene2))
cuda.memcpy_htod(gene2_cuda, gene2)


func = mod.get_function("SmithWaterman")  # priprema kernela za izvrsavanje

func(grid_cuda, gene1_cuda, gene2_cuda, np.int32(mtch), np.int32(miss), np.int32(gap),np.int32(gridSize),
     block=(gridSize-1,1,1), grid=(1, 1, 1))  # izrvrsavanje kernela

cuda.memcpy_dtoh(grid, grid_cuda)  # kopiranje Device TO Host

print("SmithWaterman:\n", grid)

SmithWaterman:
 [[0 0 0 0 0 0]
 [0 1 0 0 0 1]
 [0 0 0 1 0 0]
 [0 0 1 0 2 0]
 [0 1 0 0 0 3]
 [0 1 0 0 0 1]]


Drugi zadatak

In [ ]:
mod = SourceModule("""
     __global__ void reduce(int *a, int *result){
     
       extern __shared__ float data[];
       int idx = threadIdx.x;
       
       data[idx] = a[idx];
       __syncthreads();
       
       for(unsigned int s=1; s < blockDim.x; s *= 2) { 
       
          if (idx % (2*s) == 0  && data[idx+s]>data[idx]){
            data[idx] = data[idx + s]; 
          }   
          __syncthreads(); 
       }
       
       // write result for this block to global mem
       if (idx == 0) result[0] = data[0];

    }
   """)


reduce_func = mod.get_function("reduce")
a_cuda = to_cuda(grid)[0]

result = np.zeros(1, dtype=np.int32)
result_cuda = to_cuda(result)[0]

length = gridSize*gridSize
reduce_func(a_cuda, result_cuda, shared=grid.nbytes, 
                                      block=(length, 1, 1), 
                                      grid=(1, 1, 1))

cuda.memcpy_dtoh(result, result_cuda)

print(result)

[3]


Konstantna memorija

In [ ]:
mod = SourceModule("""
     __constant__  char geneRow[6];
     __constant__  char geneCol[6];

     __global__ void SmithWatermanCM(int *grid, int match, int miss, int gap, int n)
     {
       int tid = threadIdx.x + 1;   //thread id 1-indexed
       int cnt = 1;             //counts num of cells that thread has processed (thread should procces n cells)

       int start = 1;
       int end = 2*(n-1);

      for(int did = start; did < end; did++) // did => diagonal id
      {
        if(did >= tid && cnt < n)
        {
          char Ai = geneRow[cnt-1];
          char Bi = geneCol[tid-1];

          int left = grid[tid*n + cnt-1];       // grid[tid][did-1];
          int up = grid[(tid-1)*n + cnt];       //grid[tid-1][did];
          int top = grid[(tid-1)*n + cnt-1];    // grid[tid-1][cnt-1];

          int score = (Ai == Bi) ? match : -1*miss;

          int ans = 0;                          //answer for grid[tid][did]
          if(top + score > ans) ans = top + score;
          if(up - gap > ans) ans = up - gap;
          if(left - gap > ans) ans = left - gap;

          grid[tid*n + cnt] = ans;

          cnt++;
        }

        __syncthreads();
      }


     }
   """)

constant_mem = mod.get_function("SmithWatermanCM")

geneRow_cuda = mod.get_global('geneRow')[0]
geneCol_cuda = mod.get_global('geneCol')[0]

cuda.memcpy_htod(geneRow_cuda, gene1)
cuda.memcpy_htod(geneCol_cuda, gene2)

gridCM = np.zeros(shape=(gridSize,gridSize)).astype(dtype=np.int32)
constant_mem(grid_cuda, np.int32(mtch), np.int32(miss), np.int32(gap),np.int32(gridSize),
     block=(gridSize-1,1,1), grid=(1, 1, 1))  # izrvrsavanje kernela

cuda.memcpy_dtoh(gridCM, grid_cuda)  # kopiranje Device TO Host

print("SmithWaterman CONTANT MEMORY:\n", gridCM)

SmithWaterman CONTANT MEMORY:
 [[0 0 0 0 0 0]
 [0 1 0 0 0 1]
 [0 0 0 1 0 0]
 [0 0 1 0 2 0]
 [0 1 0 0 0 3]
 [0 1 0 0 0 1]]


Deljena memorija

In [ ]:
mod = SourceModule("""

     __global__ void SmithWatermanSharedMem(int *grid, char *geneRow, char *geneCol, int match, int miss, int gap, int n)
     {
       int tid = threadIdx.x + 1;   //thread id 1-indexed
       int cnt = 1;             //counts num of cells that thread has processed (thread should procces n cells)

       int start = 1;
       int end = 2*(n-1);
      __shared__  char geneRow_share[6];
      __shared__  char geneCol_share[6];

      geneRow_share[tid-1] = geneRow[tid-1];
      geneCol_share[tid-1] = geneCol[tid-1];

      __syncthreads();

      for(int did = start; did < end; did++) // did => diagonal id
      {
        if(did >= tid && cnt < n)
        {
          char Ai = geneRow_share[cnt-1];
          char Bi = geneCol_share[tid-1];

          int left = grid[tid*n + cnt-1];       // grid[tid][did-1];
          int up = grid[(tid-1)*n + cnt];       //grid[tid-1][did];
          int top = grid[(tid-1)*n + cnt-1];    // grid[tid-1][cnt-1];

          int score = (Ai == Bi) ? match : -1*miss;

          int ans = 0;                          //answer for grid[tid][did]
          if(top + score > ans) ans = top + score;
          if(up - gap > ans) ans = up - gap;
          if(left - gap > ans) ans = left - gap;

          grid[tid*n + cnt] = ans;

          cnt++;
        }

        __syncthreads();
      }


     }
   """)

shared_mem = mod.get_function("SmithWatermanSharedMem")

gene1_cuda = cuda.mem_alloc(len(gene1))
cuda.memcpy_htod(gene1_cuda, gene1)
gene2_cuda = cuda.mem_alloc(len(gene2))
cuda.memcpy_htod(gene2_cuda, gene2)

gridSM = np.zeros(shape=(gridSize,gridSize)).astype(dtype=np.int32)
shared_mem(grid_cuda, gene1_cuda, gene2_cuda, np.int32(mtch), np.int32(miss), np.int32(gap),np.int32(gridSize),
     block=(gridSize-1,1,1), grid=(1, 1, 1))  # izrvrsavanje kernela

cuda.memcpy_dtoh(gridSM, grid_cuda)  # kopiranje Device TO Host

print("SmithWaterman SHARED MEMORY:\n", gridSM)

SmithWaterman SHARED MEMORY:
 [[0 0 0 0 0 0]
 [0 1 0 0 0 1]
 [0 0 0 1 0 0]
 [0 0 1 0 2 0]
 [0 1 0 0 0 3]
 [0 1 0 0 0 1]]


Treci zadatak

In [ ]:
mod = SourceModule("""
     __device__  int visited[36] = {1};


     __global__ void SmithWatermanGrid(int *grid, int *vis, char *geneRow, char *geneCol, int match, int miss, int gap, int n)
     {
       int tid = threadIdx.x + blockDim.x*blockIdx.x + 1;   //thread id 1-indexed
       int cnt = 1;             //counts num of cells that thread has processed (thread should procces n cells)

       int start = 1;
       int end = 2*(n-1);

       if(tid>=n)return;
    
      for(int did = start; did < end; did++) // did => diagonal id
      {
        if(did >= tid && cnt < n)
        {
          char Ai = geneRow[cnt-1];
          char Bi = geneCol[tid-1];
          visited[tid*n+cnt] = 0;

          while(true) {
            if( visited[tid*n + cnt-1]!=0 && visited[(tid-1)*n + cnt]!=0 && visited[(tid-1)*n + cnt-1]!=0)break;
          }

          int left = grid[tid*n + cnt-1];       // grid[tid][did-1];
          int up = grid[(tid-1)*n + cnt];       //grid[tid-1][did];
          int top = grid[(tid-1)*n + cnt-1];    // grid[tid-1][cnt-1];

          int score = (Ai == Bi) ? match : -1*miss;

          int ans = 0;                          //answer for grid[tid][did]
          if(top + score > ans) ans = top + score;
          if(up - gap > ans) ans = up - gap;
          if(left - gap > ans) ans = left - gap;

          //grid[tid*n + cnt] = ans;
          atomicAdd(&grid[tid*n + cnt], ans);
          //vis[tid*n + cnt] = 1;
          atomicAdd(&visited[tid*n + cnt], 1);
          //vis[tid*n+cnt] = tid;
          cnt++;
        }
        __syncthreads();

      }
      //grid[tid] = tid*(-1);

     }
   """)



visited = np.zeros(shape=(gridSize,gridSize)).astype(dtype=np.int32)
for i in range(gridSize):
  visited[i][0]=1
  visited[0][i]=1


visited_cuda = to_cuda(visited)[0]


print(visited)

swGrid = mod.get_function("SmithWatermanGrid")  # priprema kernela za izvrsavanje
gridSW = np.zeros(shape=(gridSize,gridSize)).astype(dtype=np.int32)
gridSW_cuda = to_cuda(gridSW)[0]

swGrid(gridSW_cuda, visited_cuda, gene1_cuda, gene2_cuda, np.int32(mtch), np.int32(miss), np.int32(gap), np.int32(gridSize),
     block=(2,1,1), grid=(4, 1, 1))  # izrvrsavanje kernela

cuda.memcpy_dtoh(gridSW, gridSW_cuda)  # kopiranje Device TO Host
cuda.memcpy_dtoh(visited, visited_cuda)

print(visited)

print("SmithWatermanGRID:\n", gridSW)

/usr/local/lib/python3.8/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: device_allocation in out-of-thread context could not be cleaned up
  globals().clear()


[[1 1 1 1 1 1]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]]
[[1 1 1 1 1 1]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]]
SmithWatermanGRID:
 [[0 0 0 0 0 0]
 [0 1 0 0 0 1]
 [0 0 0 1 0 0]
 [0 0 1 0 2 0]
 [0 1 0 0 0 3]
 [0 1 0 0 0 1]]
